# Introdução ao pacote de inversão

## Regressão linear básica

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(5, 20, 100)
a, b = 0.1, 500
y = a*x + b

In [ ]:
type(x)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y, '.k')

In [ ]:
from fatiando import utils

In [ ]:
y_real = utils.contaminate(y, 0.2)

In [ ]:
plt.plot(x, y_real, '.k')

In [ ]:
from fatiando.inversion import Misfit

In [ ]:
class Regressao(Misfit):
    
    def __init__(self, x, y):
        super(Regressao, self).__init__(data=y, nparams=2, islinear=True)
        self.x = x
        
    def predicted(self, p):
        a, b = p
        return a*self.x + b
    
    def jacobian(self, p):
        A = np.empty((self.ndata, self.nparams))
        A[:, 0] = self.x
        A[:, 1] = 1
        return A

In [ ]:
reg = Regressao(x, y_real)

In [ ]:
reg

In [ ]:
reg.fit()

In [ ]:
reg.estimate_

In [ ]:
reg.predicted()

In [ ]:
plt.plot(x, y_real, '.k')
plt.plot(x, reg.predicted(), '-r')

## Desafio: Implemente uma classe que faz o ajuste de uma parábola

In [ ]:
a, b, c = 0.01, 0.1, 100
y2 = a*x**2 + b*x + c
y2_real = utils.contaminate(y2, 0.1)

In [ ]:
plt.plot(x, y2, '.k')

In [ ]:
class Parabola(Misfit):
    
    def __init__(self, x, y):
        super(Parabola, self).__init__(data=y, nparams=3, islinear=True)
        self.x = x
        
    def predicted(self, p):
        a, b, c = p
        return a*self.x**2 + b*self.x + c
    
    def jacobian(self, p):
        A = np.empty((self.ndata, self.nparams))
        A[:, 0] = self.x**2
        A[:, 1] = self.x
        A[:, 2] = 1
        return A

In [ ]:
par = Parabola(x, y2_real).fit()
par.estimate_

In [ ]:
plt.plot(x, y2_real, '.k')
plt.plot(x, par.predicted(), '-r')

## Optimização não-linear

In [ ]:
par.config

In [ ]:
par.config('newton', initial=[0, 0, 0])

In [ ]:
par.fit()

In [ ]:
par.estimate_

In [ ]:
par.stats_

In [ ]:
par.config('acor', bounds=[0, 1, 0, 1, 0, 1000]).fit()

In [ ]:
par.estimate_

## Desafio: Inversão não-linear para ajustar uma gaussiana

In [ ]:
a, b, c = 100, 0.1, -10
x = np.linspace(0, 20, 100)
y3 = a*np.exp(-b*(x + c)**2)
y3_real = utils.contaminate(y3, 5)

In [ ]:
plt.plot(x, y3_real, '.k')

In [ ]:
class Gaussian(Misfit):
    
    def __init__(self, x, y):
        super(Gaussian, self).__init__(data=y, nparams=3, islinear=False)
        self.x = x
        
    def predicted(self, p):
        a, b, c = p
        return a*np.exp(-b*(self.x + c)**2)
    
    def jacobian(self, p):
        a, b, c = p                
        A = np.zeros((self.ndata, self.nparams))  
        var = self.x + c 
        exponential = np.exp(-b*var**2) 
        A[:, 0] = exponential
        A[:, 1] = -a*exponential*(var**2)
        A[:, 2] = -a*exponential*2*b*var
        return A

In [ ]:
gauss = Gaussian(x, y3_real)

In [ ]:
gauss.config('levmarq', initial=[1, 1, -1]).fit()

In [ ]:
gauss.estimate_

In [ ]:
plt.plot(x, y3_real, '.k')
plt.plot(x, gauss.predicted(), '-r')

In [ ]:
gauss.config('acor', bounds=[1, 1000, 1e-5, 5, -100, 0]).fit()

In [ ]:
gauss.estimate_

In [ ]:
plt.plot(x, y3_real, '.k')
plt.plot(x, gauss.predicted(), '-r')